In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import glob
import os
import xarray as xr
import matplotlib.pyplot as plt

# import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

#### Load lidar data to get 'mask'

In [2]:
fname = '~/Documents/research/sierra/data/compiled_SUPERsnow.nc' #BSU
ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()

#### Load terrain layers

In [4]:
path = '/Users/meganmason491/Documents/research/sierra/data/terrain/*.nc' #BSU
terrain=xr.open_mfdataset(glob.glob(path), 
                          concat_dim=None, 
                          combine='nested', 
                          chunks={'x':1000, 'y':1000}, 
                          parallel=True).rename({'Band1':'hillshade'}).drop('transverse_mercator') #combine='nested', 
terrain=terrain.where(ds.mask==1)
terrain.close()
terrain

MergeError: conflicting values for variable 'veg' on objects to be combined. You can skip this check by specifying compat='override'.

#### Proof that terrain layers are upside down. It's a python thing. Terrain layers are perfectly fine in ncviewer

In [4]:
# plt.imshow(terrain.mask) #terrain.aspect, terrain.slope, etc.

#### flip each data variable according to the correct axis.
Result is DataArray

In [5]:
h=np.flip(terrain.hillshade,0)
a=np.flip(terrain.aspect,0)
s=np.flip(terrain.slope,0)

#### Merge flipped DataArrays into new DataSet

In [6]:
t=xr.merge([h,a,s])

#### Show that 't' has been flipped correctly

In [22]:
# plt.imshow(t.slope)

#### SAVE OUTPUT

In [20]:
t

<xarray.Dataset>
Dimensions:    (x: 17569, y: 17002)
Coordinates:
  * y          (y) float64 4.23e+06 4.23e+06 4.23e+06 ... 4.179e+06 4.179e+06
  * x          (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
    mask       (y, x) uint8 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    dem        (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    veg        (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
Data variables:
    hillshade  (y, x) float32 dask.array<chunksize=(2, 1000), meta=np.ndarray>
    aspect     (y, x) float32 dask.array<chunksize=(2, 1000), meta=np.ndarray>
    slope      (y, x) float32 dask.array<chunksize=(2, 1000), meta=np.ndarray>

In [21]:
#uncomment when you want to save:
# t.to_netcdf('~/Documents/research/sierra/data/terrain.nc')